In [1]:
from secret_key import GROQCLOUD_API
from langchain_groq import ChatGroq
llm=ChatGroq(
    model_name="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key=GROQCLOUD_API
)

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-46630?from=job%20search%20funnel")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
page_data=loader.load().pop().page_content
print(page_data)

Apply for Senior Machine Learning Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Machine Learning EngineerGdansk, PomeranianBecome a Part of the NIKE, I

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
     """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

#Chaining
chain_extract=prompt_extract|llm

In [5]:
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Machine Learning Engineer",
  "experience": "Experience in the field of ML Engineering or Software Engineering with a comparable bachelor’s degree in a relevant field, like computer science or software engineering, or equivalent experience",
  "skills": [
    "Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production",
    "Ability to significantly communicate, written, orally, and visually technical topics with peers and articulate the benefits and tradeoffs of various solutions",
    "Ability to write robust, maintainable, and extendable code in Python",
    "Experience working in and/or collaborating with a partial or fully distributed team",
    "Strong experiential understanding of data structures, algorithms, and data solutions",
    "Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks",
    "Experience with cloud architecture and techn

In [6]:
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
print(json_res)

{'role': 'Senior Machine Learning Engineer', 'experience': 'Experience in the field of ML Engineering or Software Engineering with a comparable bachelor’s degree in a relevant field, like computer science or software engineering, or equivalent experience', 'skills': ['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production', 'Ability to significantly communicate, written, orally, and visually technical topics with peers and articulate the benefits and tradeoffs of various solutions', 'Ability to write robust, maintainable, and extendable code in Python', 'Experience working in and/or collaborating with a partial or fully distributed team', 'Strong experiential understanding of data structures, algorithms, and data solutions', 'Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks', 'Experience with cloud architecture and technologies (preferably Amazon Web Services: ECR

In [8]:
type(json_res)

dict

In [9]:
import pandas as pd
df=pd.read_csv("portfolio.csv")

In [10]:
df

,TechStack,Link
0,"Computer Vision,YOLO",https://github.com/Hariprasaadh/DocScanner
1,"Deep Learning, App Development",https://github.com/Hariprasaadh/weed_detection...
2,"Machine Learning, Python",https://github.com/Hariprasaadh/Housing_Prices...
3,My Profile,https://www.linkedin.com/in/hariprasaadh-k-a54...
4,"Tensorflow,Scikit-Learn",https://github.com/Hariprasaadh/Titanic-Surviv...
5,"Web, App",https://github.com/mithilgirish/Crop-Core_Tech
6,"data structures, algorithms",https://leetcode.com/u/Hariprasaadh_K/


In [11]:
import uuid
import chromadb
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(
            documents=row["TechStack"],
            metadatas={"links":row["Link"]},
            ids=[str(uuid.uuid4())]
        )

In [12]:
links=collection.query(query_texts=["experience in python","Expertise in Computer Vision"],n_results=2).get('metadatas',[])
print(links)

[[{'links': 'https://github.com/Hariprasaadh/Housing_Prices-Advanced_Regression_Techniques'}, {'links': 'https://github.com/Hariprasaadh/Titanic-Survival-Prediction'}], [{'links': 'https://github.com/Hariprasaadh/DocScanner'}, {'links': 'https://github.com/Hariprasaadh/Housing_Prices-Advanced_Regression_Techniques'}]]


In [13]:
job=json_res
job['skills']

['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
 'Ability to significantly communicate, written, orally, and visually technical topics with peers and articulate the benefits and tradeoffs of various solutions',
 'Ability to write robust, maintainable, and extendable code in Python',
 'Experience working in and/or collaborating with a partial or fully distributed team',
 'Strong experiential understanding of data structures, algorithms, and data solutions',
 'Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks',
 'Experience with cloud architecture and technologies (preferably Amazon Web Services: ECR, SageMaker, Lambda, API Gateway)',
 'Familiarity with pipeline orchestration tools such as AirFlow',
 'Awareness of CI/CD pipelines and containerization is preferred',
 'Fluency in the application of open-source technologies and the potential of standardized p

In [14]:
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas',[])
print(links)

[[{'links': 'https://github.com/Hariprasaadh/Housing_Prices-Advanced_Regression_Techniques'}, {'links': 'https://github.com/Hariprasaadh/weed_detection_app'}], [{'links': 'https://github.com/mithilgirish/Crop-Core_Tech'}, {'links': 'https://github.com/Hariprasaadh/DocScanner'}], [{'links': 'https://github.com/Hariprasaadh/Housing_Prices-Advanced_Regression_Techniques'}, {'links': 'https://leetcode.com/u/Hariprasaadh_K/'}], [{'links': 'https://github.com/mithilgirish/Crop-Core_Tech'}, {'links': 'https://github.com/Hariprasaadh/weed_detection_app'}], [{'links': 'https://leetcode.com/u/Hariprasaadh_K/'}, {'links': 'https://github.com/Hariprasaadh/Housing_Prices-Advanced_Regression_Techniques'}], [{'links': 'https://github.com/Hariprasaadh/Titanic-Survival-Prediction'}, {'links': 'https://github.com/Hariprasaadh/weed_detection_app'}], [{'links': 'https://github.com/mithilgirish/Crop-Core_Tech'}, {'links': 'https://github.com/Hariprasaadh/weed_detection_app'}], [{'links': 'https://github.co

In [15]:
prompt_email=PromptTemplate.from_template(
     """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are K. Hariprasaadh, a passionate and dedicated candidate pursuing B.Tech in CSE with a specialization in AI and ML at VIT University, Chennai. 
        You have a strong academic foundation with a CGPA of 9.61, excellent programming skills in Python, C++, and Java, and extensive experience in 
        machine learning, computer vision, and competitive programming. You have also actively participated and won multiple hackathons and have 
        successfully developed projects like a crop management web app and educational technology tools.

        Based on the job description provided, your task is to write a professional and persuasive email expressing your enthusiasm to join the 
        company Nike. Highlight how your academic achievements, technical expertise, and project experience align with the company's goals. 
        Mention your key accomplishments and how your skills can contribute effectively to the role and the company’s mission.
        Also add the most relevant ones from the following links to showcase portfolio and add my profile link also: {link_list}

        Ensure the tone is formal, enthusiastic, and customized to the company's values and the role requirements. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)

chain_email=prompt_email|llm
res=chain_email.invoke({"job_description":str(job),"link_list":links})

In [16]:
print(res)

content="Subject: Application for Senior Machine Learning Engineer Role at Nike\n\nDear Hiring Manager,\n\nI am thrilled to apply for the Senior Machine Learning Engineer position at Nike, where I can leverage my passion for machine learning, computer vision, and software engineering to drive innovation and excellence. As a dedicated and passionate individual with a strong academic foundation in Computer Science and Engineering with a specialization in AI and ML from VIT University, Chennai, I am confident that my skills and experience align with the company's goals.\n\nWith a CGPA of 9.61, I have consistently demonstrated academic excellence and a strong understanding of programming languages such as Python, C++, and Java. My extensive experience in machine learning, computer vision, and competitive programming has equipped me with the skills to develop sophisticated analytics systems that can directly impact business outcomes. I am excited about the prospect of working on a cross-dis